In [1]:
from app import Errors
import itertools
import pandas as pd
import os
from math import pi
from datetime import timedelta

In [2]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import show, output_notebook, figure
from bokeh.palettes import Category20c
from bokeh import palettes
from bokeh import layouts
from bokeh.transform import cumsum
output_notebook()

Loading BokehJS ...

In [28]:
raw_data = Errors().error_heatmap()

In [29]:
error_heatmap = raw_data
error_heatmap['location'] = error_heatmap.apply(lambda row: f"{os.path.basename(row['filename'])}:{row['line_number']}", axis=1)
error_heatmap = error_heatmap.drop(columns=['filename', 'line_number'])
error_heatmap

error_count  end_of_day         location
device        date                                                
PTL_RD_AT_003 2020-02-24            3  2020-02-25  Database.py:111
PTL_RD_AT_000 2020-02-24            1  2020-02-25  Database.py:104
              2020-02-24            6  2020-02-25  Database.py:111
              2020-02-24            5  2020-02-25  Database.py:113
              2020-02-24            1  2020-02-25  Database.py:117
...                               ...         ...              ...
PTL_RD_ES_008 2020-03-13            2  2020-03-14  Database.py:111
PTL_RD_ES_009 2020-03-13            1  2020-03-14  Database.py:111
PTL_RD_ES_012 2020-03-13            2  2020-03-14  Database.py:111
PTL_RD_ES_006 2020-03-13            1  2020-03-14  watchdog.py:282
              2020-03-13            1  2020-03-14  watchdog.py:301

[153 rows x 3 columns]

In [143]:
def color_palette(size):
    palette_generator = itertools.cycle(palettes.Category20[20] + palettes.Set3[12] + palettes.Category20b[20])
    c = [color for color, _ in zip(palette_generator, range(num_locations))]
    return c

In [144]:
locations = pd.DataFrame(error_heatmap.location.unique(), columns=['location'])
locations['colors'] = palette(len(locations.location))
locations

,location,colors
0,Database.py:111,#1f77b4
1,Database.py:104,#aec7e8
2,Database.py:113,#ff7f0e
3,Database.py:117,#ffbb78
4,base_events.py:1285,#2ca02c
5,logging_test.py:53,#98df8a
6,Serial.py:132,#d62728
7,watchdog.py:95,#ff9896
8,watchdog.py:129,#9467bd
9,Lighting.py:284,#c5b0d5


In [145]:
eh = error_heatmap.reset_index()
errors_by_day = eh.groupby(['device', 'date']).sum().rename(columns=dict(error_count='errors_by_day'))
eh = eh.join(errors_by_day, on=['device', 'date'])
eh['error_count_normalized'] = eh.error_count / eh.errors_by_day
eh = eh.merge(locations, on = ['location'])
eh = eh.set_index(['device', 'date', eh.index]).sort_index()
eh

error_count  end_of_day         location  \
device        date                                                       
PTL_LT_AT_001 2020-03-09 94            17  2020-03-10     Serial.py:44   
PTL_RD_AT_000 2020-02-24 1              6  2020-02-25  Database.py:111   
                         74             1  2020-02-25  Database.py:104   
                         75             5  2020-02-25  Database.py:113   
                         76             1  2020-02-25  Database.py:117   
...                                   ...         ...              ...   
PTL_RD_ES_012 2020-03-12 126            2  2020-03-13  watchdog.py:261   
                         146            2  2020-03-13  watchdog.py:301   
              2020-03-13 73             2  2020-03-14  Database.py:111   
                         151            5  2020-03-14    Serial.py:155   
                         152            5  2020-03-14     Serial.py:50   

                              errors_by_day  error_count_normalized   colors  
device        date                                                            
PTL_LT_AT_001 2020-03-09 94              17                1.000000  #7f7f7f  
PTL_RD_AT_000 2020-02-24 1               15                0.400000  #1f77b4  
                         74              15                0.066667  #aec7e8  
                         75              15                0.333333  #ff7f0e  
                         76              15                0.066667  #ffbb78  
...                                     ...                     ...      ...  
PTL_RD_ES_012 2020-03-12 126             75                0.026667  #dbdb8d  
                         146             75                0.026667  #9edae5  
              2020-03-13 73              12                0.166667  #1f77b4  
                         151             12                0.416667  #ffffb3  
                         152             12                0.416667  #bebada  

[153 rows x 6 columns]

In [164]:
dates = eh.index.get_level_values(1).unique().sort_values()
x_range = min(dates) - timedelta(hours=12), max(dates) + timedelta(hours=12)

In [165]:
figures = {}

for device in eh.index.get_level_values(0).unique():
    p = figure(plot_height=300, plot_width=500, title=f"Error heatmap for {device}", toolbar_location=None,
               tools="hover", tooltips="@location: @error_count Errors", x_axis_type='datetime', x_range=x_range)

    device_data = eh.loc[device]
    for date in device_data.index.get_level_values(0).unique():
        data_source = ColumnDataSource(device_data.loc[date])
        p.vbar(bottom=cumsum('error_count_normalized', include_zero=True),
               top=cumsum('error_count_normalized'),
               x=date, width=timedelta(days=1)/2, source=data_source, fill_color='colors')

    figures[device] = p

In [166]:
show(layouts.column(*list(figures.values())))

# Compute time intervals where the ptl was online

In [164]:
from app import db
from db import DeadManPackage
import pandas as pd
from datetime import timedelta

In [225]:
query = db.session.query(DeadManPackage.device, DeadManPackage.timestamp)

In [226]:
colors = ['#00cc00', '#ff0000']

In [397]:
data = pd.DataFrame(query.all())
# compute the time difference between two consecutive rows
data['delay'] = data.groupby('device').timestamp.diff()
# The first row for each device has no 'delay' value. 
# We assume this is the start of a connected interval. 
# To do this we set the delay to be bigger than the threshold
# for dataloss
data = data.fillna(timedelta(minutes=2))
# A dataloss happened, if the delay is bigger than a threshold
# If the row 'connected' contains 1 then no data loss happend at this time
data['data_loss'] = (data.delay <= timedelta(seconds=90)).astype(int)
# compute the difference of consecutive data_loss values to see where
# the connection status changed
data['keep_row'] = data.groupby('device').data_loss.diff(periods=-1)
# We merge intervals of consecutive 'data loss' or 'connection' intervals by
# keeping only rows where the value of 'keep_rows' is not zero
data = data[data.keep_row != 0.0]
# the timestamp row now marks the beginning of an interval where
# the connection status did not change.
data = data.rename(columns=dict(timestamp='begin'))
# compute the duration of each interval
data['duration'] = data.groupby('device').begin.diff(periods=-1).abs()
data['end'] = data.begin + data.duration
data['connected'] = 1 - data.data_loss
data = data.drop(columns=['delay', 'keep_row', 'data_loss'])
data = data.sort_index()
data['color'] = colors[0]
data.loc[data.connected == 0, 'color'] = '#ff0000'

In [367]:
from datetime import datetime

In [375]:
# since the end of the last connected interval is still in the future
# it contains a NaT value. We set this to the last received timestamp for
# each device in the data set.
#end_of_last_interval = data_raw.groupby('device').timestamp.max()

In [398]:
#data.loc[data.end.isna(), 'end'] = end_of_last_interval
#data = data.reset_index()
data = data.dropna()
data = data.set_index(['device', data.index]).sort_index()
#data.duration = data.end - data.begin
data

begin               duration  \
device                                                                  
PTL_RD_AT_000 80    2020-03-12 12:25:35.196265 0 days 03:03:33.452463   
              994   2020-03-12 15:29:08.648728 0 days 00:01:53.820413   
              996   2020-03-12 15:31:02.469141 0 days 09:28:45.484429   
              10298 2020-03-13 00:59:47.953570 0 days 00:02:14.093244   
              10327 2020-03-13 01:02:02.046814 0 days 11:41:09.469620   
              21385 2020-03-13 12:43:11.516434 0 days 00:02:45.886590   
              21419 2020-03-13 12:45:57.403024 0 days 12:13:16.397983   
              33790 2020-03-14 00:59:13.801007 0 days 00:02:47.392036   
              33818 2020-03-14 01:02:01.193043 1 days 19:08:48.584410   
PTL_RD_AT_001 0     2020-03-12 11:01:27.834999 0 days 01:14:13.202586   
              74    2020-03-12 12:15:41.037585 0 days 00:02:36.370863   
              75    2020-03-12 12:18:17.408448 0 days 00:02:00.619397   
              77    2020-03-12 12:20:18.027845 0 days 00:03:30.691471   
              78    2020-03-12 12:23:48.719316 0 days 03:04:32.126293   
              990   2020-03-12 15:28:20.845609 0 days 00:02:48.684818   
              997   2020-03-12 15:31:09.530427 0 days 09:28:40.386910   
              10299 2020-03-13 00:59:49.917337 0 days 00:02:11.499660   
              10324 2020-03-13 01:02:01.416997 0 days 11:26:01.288862   
              21185 2020-03-13 12:28:02.705859 0 days 00:01:56.018869   
              21209 2020-03-13 12:29:58.724728 0 days 00:14:02.952029   
              21396 2020-03-13 12:44:01.676757 0 days 00:03:23.034242   
              21428 2020-03-13 12:47:24.710999 0 days 00:33:04.698473   
              21875 2020-03-13 13:20:29.409472 0 days 00:02:31.282048   
              21915 2020-03-13 13:23:00.691520 2 days 06:48:15.916526   
PTL_RD_AT_002 82    2020-03-12 12:25:51.745822 0 days 03:03:33.586009   
              995   2020-03-12 15:29:25.331831 0 days 00:01:53.143676   
              998   2020-03-12 15:31:18.475507 0 days 09:28:44.515759   
              10309 2020-03-13 01:00:02.991266 0 days 00:01:55.920917   
              10314 2020-03-13 01:01:58.912183 0 days 11:43:08.175419   
              21410 2020-03-13 12:45:07.087602 0 days 00:02:25.837635   
...                                        ...                    ...   
PTL_RD_ES_010 57202 2020-03-15 00:02:11.486069 0 days 00:01:57.607735   
              57222 2020-03-15 00:04:09.093804 0 days 20:06:39.961746   
PTL_RD_ES_011 1021  2020-03-12 15:33:15.211378 0 days 09:26:38.021107   
              10304 2020-03-13 00:59:53.232485 0 days 00:02:05.891761   
              10316 2020-03-13 01:01:59.124246 0 days 09:42:43.268265   
              21439 2020-03-13 10:44:42.392511 0 days 02:06:27.879794   
              21469 2020-03-13 12:51:10.272305 0 days 11:11:02.383391   
              32840 2020-03-14 00:02:12.655696 0 days 00:01:58.571545   
              32858 2020-03-14 00:04:11.227241 0 days 00:55:10.543565   
              33793 2020-03-14 00:59:21.770806 0 days 00:02:38.687015   
              33816 2020-03-14 01:02:00.457821 0 days 23:00:12.518264   
              57205 2020-03-15 00:02:12.976085 0 days 00:01:58.520171   
              57228 2020-03-15 00:04:11.496256 0 days 20:06:38.807265   
PTL_RD_ES_012 1024  2020-03-12 15:33:25.502119 0 days 00:41:06.323346   
              2224  2020-03-12 16:14:31.825465 0 days 00:52:09.410872   
              2274  2020-03-12 17:06:41.236337 0 days 00:46:06.639427   
              3597  2020-03-12 17:52:47.875764 0 days 00:58:06.842775   
              3627  2020-03-12 18:50:54.718539 0 days 06:09:07.990440   
              10310 2020-03-13 01:00:02.708979 0 days 00:01:59.225369   
              10330 2020-03-13 01:02:01.934348 0 days 11:37:05.224913   
              21340 2020-03-13 12:39:07.159261 0 days 00:02:36.013582   
              21362 2020-03-13 12:41:43.172843 0 days 00:06:01.207255   
              21435 2020-0

In [399]:
device = 'PTL_RD_ES_012'
device_data = data.loc[device]
data_source = ColumnDataSource(device_data)
xrange = device_data.begin.min(), device_data.end.max()
fig = figure(x_axis_type="datetime", x_range=x_range, y_range=(0, 0.25),
             plot_height=50, plot_width=800)
fig.hbar(y=0.125, left='begin', right='end', height=0.25, color='color', source=data_source)
fig.yaxis.visible=False
fig.toolbar.logo = None
fig.toolbar_location = None
show(fig)